In [1]:
# # general imports 
import os
from pathlib import Path
import numpy as np
import pandas as pd 

# nwb specific imports 
import pynwb
from nwbwidgets import nwb2widget
from hdmf_zarr import NWBZarrIO 

In [2]:
# Load metadata CSV 

# metadata = pd.read_csv('/data/metadata/V1DD_metadata.csv') # V1DD 
metadata = pd.read_csv('/data/metadata/dynamic_foraging_metadata.csv') # Dynamic Foraging 
# metadata = pd.read_csv('/data/metadata/bci_metadata.csv') # BCI 
metadata.head()

,_id,name,subject_id,genotype,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,trials_total,trials_rewarded,quality_control
0,8bcbd349-2ac7-467a-8ccd-f05b2319f06d,behavior_761432_2025-03-03_11-51-23_processed_...,761432,Dbh-Cre-KI/wt,2024-08-21,Male,Coupled Baiting,2025-03-03T11:51:23.846831-08:00,['The behavior auditory go cue'],Behavior Platform,['Behavior'],[613],[246],"['Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']"
1,d9dce157-83e7-4d79-8c42-c48a73d7849b,behavior_761433_2025-03-03_11-47-13_processed_...,761433,Dbh-Cre-KI/wt,2024-08-21,Male,Coupled Without Baiting,2025-03-03T11:47:13.392064-08:00,['The behavior auditory go cue'],Behavior Platform,['Behavior'],[509],[209],"['Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']"
2,ed9f660f-0990-427f-84cf-5d454331c348,behavior_770803_2025-02-28_13-11-33_processed_...,770803,Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-I...,2024-10-07,Male,Uncoupled Baiting,2025-02-28T13:11:33.083835-08:00,['The behavior auditory go cue'],Behavior Platform,['Behavior'],[539],[261],"['Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']"
3,c1d96ba0-2393-45b4-a835-f06d8846d8ed,behavior_781469_2025-03-03_13-26-23_processed_...,781469,wt/wt,2024-10-08,Male,Uncoupled Baiting,2025-03-03T13:26:23.690824-08:00,['The behavior auditory go cue'],Behavior Platform,['Behavior'],[403],[180],"['Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']"
4,7de28343-b4f0-466f-912c-a5792de1e28a,behavior_752014_2025-03-03_09-34-48_processed_...,752014,Dbh-Cre-KI/wt,2024-07-05,Male,Uncoupled Without Baiting,2025-03-03T09:34:48.766920-08:00,['The behavior auditory go cue'],Behavior Platform,['Behavior'],[651],[333],"['Pass', 'Pass', 'Pass', 'Pass', 'Pass', 'Pass']"


In [3]:
metadata.columns

Index(['_id', 'name', 'subject_id', 'genotype', 'date_of_birth', 'sex',
       'session_type', 'session_time', 'stimulus_epochs', 'project_name',
       'modality', 'trials_total', 'trials_rewarded', 'quality_control'],
      dtype='object')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What modality was used?
</div>

In [5]:
metadata.modality.unique()

array(["['Behavior']", "['Behavior', 'Behavior videos']"], dtype=object)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** How many mice were used in this dataset?
</div>

In [12]:
len(metadata.subject_id.unique())

67

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** How many sessions? How many sessions per mouse? 
</div>

In [13]:
len(metadata.name.unique())

429

In [8]:
metadata['subject_id'].value_counts()

subject_id
761038    20
781469    17
758016    16
775510    16
761433    15
          ..
776293     1
781162     1
788586     1
789191     1
794980     1
Name: count, Length: 67, dtype: int64

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What transgenic lines were used in this dataset? 
    
What cell types do these transgenic lines target? Refer to the [Data Book](https://allenswdb.github.io/background/transgenic-tools.html) for this. 
</div>

In [9]:
metadata.genotype.unique()

array(['Dbh-Cre-KI/wt',
       'Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-IRES2-tTA2-WPRE)-hyg/wt',
       'wt/wt', 'Drd1a-Cre/wt;Oi8(H11-CAG-Cas9)/wt',
       'Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai231(TICL-ChrimsonR-GFP-WPRE-ICF-CoChR-oScarlet-WPRE)-hyg/wt',
       'Pdyn-IRES-Cre/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Drd3-Cre_KI196/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Calcr-Cre/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Drd2-Cre_ER44/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Adora2a-Cre/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Drd1a-Cre/wt;Oi9(H11-CAG-LSL-Cas9)/wt',
       'Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai233(TICL-ChRmine-TS-GFP-WPRE-ICF-CoChR-oScarlet-WPRE)-hyg/wt',
       'VGAT-ChR2-YFP/wt', 'Sert-Cre/wt', 'Drd3-Cre_KI196/wt',
       'Calcr-Cre/wt'], dtype=object)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What structures were targeted? If not in metadata, might need to infer from Data Book or NWB file itself. 

In [14]:
# n/a 

In [17]:
# # Load session data 

nwb_path_zarr = '/data/dynamic-foraging-behavior/behavior_729678_2025-01-10_09-24-20_processed_2025-02-22_15-57-11/nwb/behavior_729678_2025-01-10_09-24-20.nwb'

In [18]:
io = NWBZarrIO(nwb_path_zarr, mode = 'r') 
nwbfile_zarr = io.read()

/opt/conda/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.7.0 because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [19]:
nwb2widget(nwbfile_zarr)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What units/neurons are available? How many neurons were recorded in this session? Has the data been pre-filtered based on quality control metrics? 

In [21]:
# n/a 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What stimuli, behaviors, and conditions were applied or recorded in this experiment? 

In [42]:
trials = nwbfile_zarr.intervals['trials'].to_dataframe()
trials

,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,...,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y1,lickspout_position_y2,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,
0,1.889993e+07,1.889993e+07,1.0,False,False,1.889993e+07,1.889993e+07,1.889993e+07,True,False,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
1,1.889993e+07,1.889994e+07,1.0,False,True,1.889994e+07,1.889994e+07,1.889994e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
2,1.889994e+07,1.889995e+07,1.0,False,True,1.889994e+07,1.889994e+07,1.889994e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
3,1.889995e+07,1.889995e+07,1.0,False,False,1.889995e+07,1.889995e+07,1.889995e+07,True,False,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
4,1.889995e+07,1.889996e+07,1.0,False,True,1.889996e+07,1.889996e+07,1.889996e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,1.890439e+07,1.890440e+07,1.0,False,True,1.890440e+07,1.890440e+07,1.890440e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
522,1.890440e+07,1.890441e+07,1.0,False,True,1.890441e+07,1.890441e+07,1.890441e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0
523,1.890441e+07,1.890442e+07,1.0,False,True,1.890441e+07,1.890441e+07,1.890441e+07,True,True,...,2.3,1.0,GRADUATED,NaN,15.6,22.0,22.0,18.0,2.0,2.0


In [43]:
trials.columns

Index(['start_time', 'stop_time', 'animal_response', 'rewarded_historyL',
       'rewarded_historyR', 'delay_start_time', 'goCue_start_time',
       'reward_outcome_time', 'bait_left', 'bait_right',
       'base_reward_probability_sum', 'reward_probabilityL',
       'reward_probabilityR', 'reward_random_number_left',
       'reward_random_number_right', 'left_valve_open_time',
       'right_valve_open_time', 'block_beta', 'block_min', 'block_max',
       'min_reward_each_block', 'delay_beta', 'delay_min', 'delay_max',
       'delay_duration', 'ITI_beta', 'ITI_min', 'ITI_max', 'ITI_duration',
       'response_duration', 'reward_consumption_duration', 'laser_on_trial',
       'laser_wavelength', 'laser_location', 'laser_power', 'laser_duration',
       'laser_condition', 'laser_condition_probability', 'laser_start',
       'laser_start_offset', 'laser_end', 'laser_end_offset', 'laser_protocol',
       'laser_frequency', 'laser_rampingdown', 'laser_pulse_duration',
       'auto_waterL', '

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: How long is a session? 

In [58]:
((trials.stop_time.iloc[len(trials)-1]) - trials.start_time.iloc[0])/60

np.float64(75.04583306666464)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What is the sampling rate? 

In [59]:
#n/a

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: Did the mice receive behavior training before recording sessions? Are the sessions longitudinal? 

Dataset encapsulates mouse's Dynamic Foraging Behavior training sessions 